In [ ]:
%%time
%%capture

!pip install langchain
!pip install langchain-openai
!pip install faiss-gpu
!pip install sentence_transformers
!pip install openai
!pip install python-dotenv

In [1]:
%%time

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader

%load_ext dotenv
%dotenv

token = os.environ.get('ONE_API_TOKEN')
base_url=os.environ.get('ONE_API_URL')

CPU times: user 632 ms, sys: 116 ms, total: 747 ms
Wall time: 709 ms


In [2]:
%%time

model_name='/models/bge-m3'
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

CPU times: user 3.57 s, sys: 1.24 s, total: 4.82 s
Wall time: 4.31 s


In [3]:
%%time
%%capture

documents=[]
loader = TextLoader("./孔乙己.txt")
documents.extend(loader.load())

CPU times: user 2.1 ms, sys: 0 ns, total: 2.1 ms
Wall time: 1.98 ms


In [4]:
%%time

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=128,chunk_overlap=20)
text_splits=text_splitter.split_documents(documents)

len(text_splits)

CPU times: user 8.08 ms, sys: 39 µs, total: 8.11 ms
Wall time: 7.67 ms


39

In [5]:
%%time

vectorstore = FAISS.from_documents(text_splits, embeddings)

CPU times: user 1.85 s, sys: 60.3 ms, total: 1.91 s
Wall time: 1.91 s


In [6]:
%%time

query = "孔乙己"
docs = vectorstore.similarity_search(query,k=10)

for doc in docs:
    display(doc.page_content)

'孔乙己⑴'

'孔乙己是站着喝酒而穿长衫的唯一的人。他身材很高大；青白脸色，皱纹间时常夹些伤痕；一部乱蓬蓬的花白的胡子。穿的虽然是长衫，可是又脏又破，似乎十多年没有补，也没有洗。他对人说话，总是满口之乎者也，教人半懂不懂的。因为他姓孔，别人便从描红纸⑵上的“上大人孔'

'孔乙己是这样的使人快活，可是没有他，别人也便这么过。'

'听人家背地里谈论，孔乙己原来也读过书，但终于没有进学⑷，又不会营生；于是愈过愈穷，弄到将要讨饭了。幸而写得一笔好字，便替人家钞钞书，换一碗饭吃。可惜他又有一样坏脾气，便是好吃懒做。坐不到几天，便连人和书籍纸张笔砚，一齐失踪。如是几次，叫他钞书的人也没'

'孔乙己喝过半碗酒，涨红的脸色渐渐复了原，旁人便又问道，“孔乙己，你当真认识字么？”孔乙己看着问他的人，显出不屑置辩的神气。他们便接着说道，“你怎的连半个秀才也捞不到呢？”孔乙己立刻显出颓唐不安模样，脸上笼上了一层灰色，嘴里说些话；这回可是全是之乎者也'

...

'为他姓孔，别人便从描红纸⑵上的“上大人孔乙己”这半懂不懂的话里，替他取下一个绰号，叫作孔乙己。孔乙己一到店，所有喝酒的人便都看着他笑，有的叫道，“孔乙己，你脸上又添上新伤疤了！”他不回答，对柜里说，“温两碗酒，要一碟茴香豆。”便排出九文大钱。他们又故意的高'

'自此以后，又长久没有看见孔乙己。到了年关，掌柜取下粉板说，“孔乙己还欠十九个钱呢！”到第二年的端午，又说“孔乙己还欠十九个钱呢！”到中秋可是没有说，再到年关也没有看见他。\n\n\u3000\u3000我到现在终于没有见——大约孔乙己的确死了。\n\n\u3000\u3000一九一九年三月。⑻'

'你知道么？”我愈不耐烦了，努着嘴走远。孔乙己刚用指甲蘸了酒，想在柜上写字，见我毫不热心，便又叹一口气，显出极惋惜的样子。'

'砚，一齐失踪。如是几次，叫他钞书的人也没有了。孔乙己没有法，便免不了偶然做些偷窃的事。但他在我们店里，品行却比别人都好，就是从不拖欠；虽然间或没有现钱，暂时记在粉板上，但不出一月，定然还清，从粉板上拭去了孔乙己的名字。'

CPU times: user 38.5 ms, sys: 4.11 ms, total: 42.6 ms
Wall time: 47 ms


In [8]:
%%time

system_message=SystemMessage(content="你是小羽，是一个人工智能助手。")

chat = ChatOpenAI(api_key=token, 
                  base_url=base_url,
                  model='xiaoyu', 
                  temperature=0,
                  streaming=True)

messages = [
    system_message,
    HumanMessage(content="孔乙己穿着打扮啥样"),
]

for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

在鲁迅先生的小说《孔乙己》中，并没有详细描述孔乙己的穿着打扮。但根据文中对他的描绘，孔乙己是一个穷困潦倒、衣衫褴褛的读书人形象，常常穿着一件破旧的长衫，满口之乎者也，且腿脚有残疾，走路时需要拿竹竿支撑。他的外貌特征和落魄状态是小说中用来反映社会底层知识分子命运的一个重要元素。CPU times: user 299 ms, sys: 29.2 ms, total: 329 ms
Wall time: 3.71 s


In [9]:
%%time

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = VectorStoreRetriever(vectorstore=vectorstore,search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=chat, retriever=retriever
)

question='孔乙己穿着打扮啥样'
qa_chain.invoke({"query": question})

CPU times: user 88.6 ms, sys: 14.4 ms, total: 103 ms
Wall time: 2.21 s


{'query': '孔乙己穿着打扮啥样',
 'result': '孔乙己身材很高大；青白脸色，皱纹间时常夹些伤痕；一部乱蓬蓬的花白的胡子。穿的虽然是长衫，可是又脏又破，似乎十多年没有补，也没有洗。'}

In [13]:
%%time

question='孔乙己写回字，有几种写法'
qa_chain.invoke({"query": question})

CPU times: user 76.2 ms, sys: 19.9 ms, total: 96.1 ms
Wall time: 2.65 s


{'query': '孔乙己写回字，有几种写法',
 'result': '回字有四种写法：回字通常只有三种写法：回、〔外“冂”内“巳”〕、〔“面”之下部〕。第四种写作〔外“囗”内“目”〕（见《康熙字典·备考》），极少见。'}

In [28]:
%%time

question='孔乙己最后欠了多少钱'
qa_chain.invoke({"query": question})

CPU times: user 40.2 ms, sys: 3.79 ms, total: 44 ms
Wall time: 1.47 s


{'query': '孔乙己最后欠了多少钱', 'result': '十九个钱'}

In [29]:
%%time

question='孔乙己因为什么被打断了腿'
qa_chain.invoke({"query": question})

CPU times: user 88.7 ms, sys: 5.92 ms, total: 94.6 ms
Wall time: 3.68 s


{'query': '孔乙己因为什么被打断了腿',
 'result': '孔乙己被打断腿的原因，据文中提到的喝酒的人所说，是因为他偷东西。具体细节是“他总仍旧是偷。这一回，是自己发昏，竟偷到丁举人家里去了。他家的东西，偷得的么？”这表明他是因盗窃而受到的惩罚。'}

In [30]:
%%time

question='孔乙己一般都点啥'
qa_chain.invoke({"query": question})

CPU times: user 56.7 ms, sys: 0 ns, total: 56.7 ms
Wall time: 1.98 s


{'query': '孔乙己一般都点啥', 'result': '孔乙己在酒店点的是酒和茴香豆，他会排九文大钱来支付。'}

In [31]:
%%time

question='孔乙己为啥不读书了'
qa_chain.invoke({"query": question})

CPU times: user 52.5 ms, sys: 16.1 ms, total: 68.6 ms
Wall time: 2.55 s


{'query': '孔乙己为啥不读书了',
 'result': '孔乙己原来也读过书，但最终没有进学⑷，可能是因为各种原因，比如家庭背景、经济困难或是个人兴趣和才能未能得到充分发挥。'}

In [10]:
%%time

question='书中的我是的是谁，和孔乙己是啥关系'
qa_chain.invoke({"query": question})

CPU times: user 52.2 ms, sys: 8.88 ms, total: 61.1 ms
Wall time: 1.04 s


{'query': '书中的我是的是谁，和孔乙己是啥关系', 'result': '书中没有提及“我”是谁，也没有直接描述“我”与孔乙己的关系。'}

In [33]:
%%time

question='《孔乙己》发表在哪里'
qa_chain.invoke({"query": question})

CPU times: user 52.1 ms, sys: 21.4 ms, total: 73.5 ms
Wall time: 2.05 s


{'query': '《孔乙己》发表在哪里', 'result': '《孔乙己》最初发表于一九一九年四月《新青年》第六卷第四号。'}

In [34]:
%%time

question='孔乙己的结局是什么？'
qa_chain.invoke({"query": question})

CPU times: user 52.3 ms, sys: 732 µs, total: 53 ms
Wall time: 1.65 s


{'query': '孔乙己的结局是什么？', 'result': '我到现在终于没有见——大约孔乙己的确死了。'}

In [35]:
%%time

question='店掌柜对孔乙己是啥态度'
qa_chain.invoke({"query": question})

CPU times: user 74.1 ms, sys: 10.6 ms, total: 84.7 ms
Wall time: 3.49 s


{'query': '店掌柜对孔乙己是啥态度',
 'result': '掌柜是一副凶脸孔，对孔乙己的态度显得冷漠且带有一丝鄙视，这可以从掌柜对孔乙己的称呼和说话的语气中看出。当提到孔乙己欠钱时，掌柜不仅再次提起，还带有嘲笑的意味。'}

In [11]:
%%time

from bge_rerank import BgeRerank
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever

compressor = BgeRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=retriever)

CPU times: user 3.15 s, sys: 1.84 s, total: 4.99 s
Wall time: 3.78 s


In [12]:
%%time

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

kc = RetrievalQA.from_llm(llm=chat, retriever=compression_retriever, prompt=prompt)

question='书中的我是的是谁，和孔乙己是啥关系'
kc.invoke({"query": question})

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacty of 3.81 GiB of which 14.00 MiB is free. Process 1574765 has 3.79 GiB memory in use. Of the allocated memory 3.70 GiB is allocated by PyTorch, and 26.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF